In [2]:
from shelpers import backends, detector, models, df_metrics, delete_representations, get_id
import cv2
from cv2 import VideoCapture
from deepface import DeepFace
import os
import sqlite3

In [3]:
from shelpers import write_to_db, get_user_info

In [4]:
def register_face(id, upload_image=None, live = False):
    result = {"success": False, "message": "", "id": id, "image_path": ""}

    # define the path to the image_database directory
    img_db_path = os.path.join(os.getcwd(), 'img_db')

    # create the database if it does not exist
    if not os.path.exists(img_db_path):
        os.mkdir(img_db_path)

    # check for duplicate id
    if id + '.jpg' in os.listdir(img_db_path):
        print("id already exists")
        result["message"] = "id already exists"
        check = input("Enter new id or enter 'x' to quit: ").lower()
        if check == 'x':
            print("exiting process...")
            result['message'] = "No face registered.... exiting process"
            return result
        else:
            id = check


    if live:
        # take in image from cam
        cap = VideoCapture(0)

        while True:
            ret, frame = cap.read()
            face = detector(frame)
            if face:
                # crop and save image
                x, y , w, h, _, _ = face[0]['facial_area'].values()
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
                crop_img = frame[y:y+h, x:x+w]

                # Display the image with the detected face
                cv2.imshow('detect face', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    result["message"] = "quit detected. No face detected in the provided image"
                    break

                if cv2.waitKey(1) & 0xFF == ord('c'):
                    # Save face
                    face_path = os.path.join(img_db_path, id + '.jpg')
                    # face_path = "/img_db/"+id+".jpg"
                    cv2.imwrite(face_path, crop_img)
                    print(f" id {id} registered successfully")
                    print(f"image stored in {face_path}")

                    delete_representations()
                    result["success"] = True
                    result["message"] = f"id {id} registered"
                    result["image_path"] = face_path
                    break

            else:
                # if no face detected
                h,w = frame.shape[:2]
                text = 'Adjust face and brightness'
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontsize = 1
                color = (0,255,0)
                thickness = 2
                text_size, _ = cv2.getTextSize(text, font, fontsize, thickness)

                x = (w - text_size[0]) // 2
                y = (h + text_size[1]) // 2
                cv2.putText(frame, text, (x, y), font, fontsize, color, thickness)
                # cv2.putText(frame, text, (10, h - 20), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0,), 1)
                # cv2.putText(crop_img, 'Align face', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                cv2.imshow('detect face', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    result["message"] = "quit detected. No face detected in the provided image"
                    break

        cv2.destroyAllWindows()
        cap.release()
        return result


    # Load the image
    else:
        # check image_path is not None
        if upload_image is None:
            result["message"] = "No image provided"
            print(result["message"])
            return result

        frame = cv2.imread(upload_image)

        # perform face detection
        face = detector(frame)
        if face:
            # crop and save image
            x, y , w, h, _, _ = face[0]['facial_area'].values()
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
            crop_img = frame[y:y+h, x:x+w]

            plt.imshow(crop_img)

            # Display the image with the detected face
            # cv2.imshow('detect face', frame)
            # cv2.waitKey(0)

            # Save face
            face_path = os.path.join(img_db_path, id + '.jpg')
            # face_path = "/img_db/"+id+".jpg"
            cv2.imwrite(face_path, crop_img)
            print(f" id {id} registered successfully")
            print(f"image stored in {face_path}")

            delete_representations()
            result["success"] = True
            result["message"] = f"id {id} registered successfully"
            result["image_path"] = face_path
            return result

        else:
            result["message"] = "No face detected in the provided image"
            print(result["message"])
            cv2.destroyAllWindows()
            return result

In [6]:
id = input("enter id: ")
upload_image = input("enter image path: ")

register_face(id, upload_image=upload_image)

No face detected in the provided image


[ WARN:0@313.005] global loadsave.cpp:241 findDecoder imread_(''): can't open/read file: check file path/integrity


{'success': False,
 'message': 'No face detected in the provided image',
 'id': '',
 'image_path': ''}

In [3]:
def register_face(id, **kwargs):
    img_path = os.getcwd() + '/img_db'
    # check if path exist
    if not os.path.exists(img_path):
        os.mkdir(img_path)
    # check for duplicate id
    if id + '.jpg' in os.listdir(img_path):
        print("id already exists")
        check = input("Enter new id or enter 'x' to quit: ").lower()
        if check == 'x':
            print("exiting command...")
            return
        else:
            id = check
            
    cap = VideoCapture(0)

    while True:
        ret, frame = cap.read()
        face = detector(frame)
        # print(face)
        if face:
            # crop and save image
            x,y,w,h,_,_ = face[0]['facial_area'].values()
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
            crop_img = frame[y:y+h, x:x+w]
            cv2.imshow('detect face', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            elif cv2.waitKey(1) & 0xFF == ord('c'):
                # save face
                face_path = img_path+"/"+id+".jpg"
                cv2.imwrite(face_path, crop_img)
                for key, value in kwargs.items():
                    info[key] = value
                # write to db
                write_to_db(id, **info)
                print(f" id {id} registered successfully")
                print(f"image stored in {face_path}")
                delete_representations()
                
                break
        else:
            # if no face detected
            h,w = frame.shape[:2]
            text = 'Adjust face and brightness'
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontsize = 1  
            color = (0,255,0)
            thickness = 2
            text_size, _ = cv2.getTextSize(text, font, fontsize, thickness)

            x = (w - text_size[0]) // 2  
            y = (h + text_size[1]) // 2
            cv2.putText(frame, text, (x, y), font, fontsize, color, thickness)
            # cv2.putText(frame, text, (10, h - 20), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0,), 1)
            # cv2.putText(crop_img, 'Align face', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow('detect face', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows() 

In [4]:
# cap = VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     face = detector(frame)
#     print(face)
#     if face:
#         feed = face
#         break

# cap.release()
# cv2.destroyAllWindows() 

In [3]:
def register_face(id, image_path=None, live = False):
    result = {"success": False, "message": "", "id": id, "image_path": ""}

    # check for duplicate id
    if id + '.jpg' in os.listdir(os.getcwd() + '/img_db/'):
        print("id already exists")
        result["message"] = "id already exists"
        check = input("Enter new id or enter 'x' to quit: ").lower()
        if check == 'x':
            print("exiting command...")
            result["message"] = "exiting command..."
            return result
        else:
            id = check

    if live:
        # take in image from cam
        cap = VideoCapture(0)

        while True:
            ret, frame = cap.read()
            face = detector(frame)
            if face:
                # crop and save image
                x, y , w, h, _, _ = face[0]['facial_area'].values()
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
                crop_img = frame[y:y+h, x:x+w]

                # Display the image with the detected face
                cv2.imshow('detect face', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    result["message"] = "quit detected. No face detected in the provided image"
                    break

                if cv2.waitKey(1) & 0xFF == ord('c'):
                    # Save face
                    face_path = os.path.join(os.getcwd() + '/img_db/', id + '.jpg')
                    # face_path = "/img_db/"+id+".jpg"
                    cv2.imwrite(face_path, crop_img)
                    print(f" id {id} registered successfully")
                    print(f"image stored in {face_path}")

                    delete_representations()
                    result["success"] = True
                    result["message"] = f"id {id} registered successfully"
                    result["image_path"] = face_path
                    break

            else:
                # if no face detected
                h,w = frame.shape[:2]
                text = 'Adjust face and brightness'
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontsize = 1
                color = (0,255,0)
                thickness = 2
                text_size, _ = cv2.getTextSize(text, font, fontsize, thickness)

                x = (w - text_size[0]) // 2
                y = (h + text_size[1]) // 2
                cv2.putText(frame, text, (x, y), font, fontsize, color, thickness)
                # cv2.putText(frame, text, (10, h - 20), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0,), 1)
                # cv2.putText(crop_img, 'Align face', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                cv2.imshow('detect face', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    result["message"] = "quit detected. No face detected in the provided image"
                    break

        cv2.destroyAllWindows()
        cap.release()
        return result


    # Load the image
    else:
        # check image_path is not None
        if image_path is None:
            result["message"] = "image_path is None"
            print(result["message"])
            return result

        frame = cv2.imread(image_path)

        # perform face detection
        face = detector(frame)
        if face:
            # crop and save image
            x, y , w, h, _, _ = face[0]['facial_area'].values()
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
            crop_img = frame[y:y+h, x:x+w]

            # Display the image with the detected face
            # cv2.imshow('detect face', frame)
            # cv2.waitKey(0)

            # Save face
            face_path = os.path.join(os.getcwd() + '/img_db/', id + '.jpg')
            # face_path = "/img_db/"+id+".jpg"
            cv2.imwrite(face_path, crop_img)
            print(f" id {id} registered successfully")
            print(f"image stored in {face_path}")

            delete_representations()
            result["success"] = True
            result["message"] = f"id {id} registered successfully"
            result["image_path"] = face_path
            return result

        else:
            result["message"] = "No face detected in the provided image"
            print(result["message"])
            cv2.destroyAllWindows()
            return result


In [4]:
image_path = './in_img/test_img.jpg'
id='mme183419'
result = register_face(id, image_path)

 id mme183419 registered successfully
image stored in c:\Users\biodu\Desktop\me\anew_work\face_recog_system/img_db/mme183419.jpg


In [5]:
result

{'success': True,
 'message': 'id mme183419 registered successfully',
 'id': 'mme183419',
 'image_path': 'c:\\Users\\biodu\\Desktop\\me\\anew_work\\face_recog_system/img_db/mme183419.jpg'}

In [5]:
# register face
info = {
    'name': 'Daniel',
    'position': 'Manager',
    'phone_number': '123-456-8890',}

register_face('mme173506', **info)

id already exists
user mme173300 info written successfully
 id mme173300 registered successfully
image stored in c:\Users\biodu\Desktop\me\anew_work\face_recog_system/img_db/mme173300.jpg


In [6]:
def face_detect():
    cap = cv2.VideoCapture(0)
    dfs = []
    
    while True:
        ret, frame = cap.read()

        if ret == False:
            break
        
        face = detector(frame)
        if face is not None:
            # face detection and recognition logic
            x,y,w,h,_,_ = face[0]['facial_area'].values()  
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255, 0), 3)
            # cv2.imshow('detect face', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('c'):
                df = DeepFace.find(img_path=frame, db_path="./img_db/", model_name=models[2], 
                                   distance_metric=df_metrics[2], enforce_detection=False)
                                #    detector_backend=backends[0])
                dfs.append(df)
                break

        cv2.imshow('display', frame)
                
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    

    user_id = get_id(dfs)
    
    # get user info by id
    user_info = get_user_info(user_id)
    # print(f"user id: {user_id}")
    
    return user_id, user_info

In [8]:
user_id, user_info = face_detect()

24-05-28 20:08:49 - Found 4 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 4/4 [00:15<00:00,  3.87s/it]


24-05-28 20:09:05 - There are now 4 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
24-05-28 20:09:05 - Searching [[[21 10  0]
  [22 10  0]
  [23 10  3]
  ...
  [ 1  2 23]
  [ 1  2 23]
  [ 9 10 31]]

 [[20 11  0]
  [18  8  0]
  [15  4  0]
  ...
  [ 0  1 19]
  [ 0  1 19]
  [ 5  7 25]]

 [[14 12  0]
  [12  9  0]
  [10  4  0]
  ...
  [ 0  0  9]
  [ 0  0 11]
  [ 2  4 15]]

 ...

 [[ 5  2  8]
  [ 4  1  6]
  [ 2  1  5]
  ...
  [ 0  4  6]
  [ 2  7  9]
  [ 4  9 11]]

 [[15 11 20]
  [11  7 16]
  [ 6  2 10]
  ...
  [ 4  5 19]
  [ 9 11 25]
  [13 16 31]]

 [[27 21 33]
  [20 14 26]
  [11  5 16]
  ...
  [ 8  6 28]
  [14 14 37]
  [19 20 43]]] in 4 length datastore
24-05-28 20:09:06 - find function duration 16.80139136314392 seconds


In [10]:
user_info

('mme173300', 'Daniel', 'Manager', '123-456-8890')

In [159]:
def check_duplicate_registration(id):
    if id in os.listdir(os.getcwd() + '/img_db/'):
        return True
    else:
        return False

In [29]:
import sqlite3

In [32]:
try:
    with sqlite3.connect('database.db') as conn:
        cur = conn.cursor()

        # create table
        cur.execute('''
            CREATE TABLE IF NOT EXISTS users(
                id INTEGER PRIMARY KEY,
                name TEXT,
                position TEXT,
                phone_number TEXT
            )
        ''')

        # insert data into the table
        cur.execute('''
            INSERT INTO users (name, position, phone_number)
            VALUES(?, ?, ?)
            ''', ('John', 'Manager', '123-456-7890'))
    
except sqlite3.Error as e:
    print(f"Error: {e}")



Error: table users has no column named position


In [34]:
import sqlite3

try:
    with sqlite3.connect('database.db') as conn:
        cur = conn.cursor()

        # Drop the table if it exists
        cur.execute('DROP TABLE IF EXISTS users')

        # create table
        cur.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                name TEXT,
                position TEXT,
                phone_number TEXT
            )
        ''')

        # insert data into the table
        cur.execute('''
            INSERT INTO users (name, position, phone_number)
            VALUES (?, ?, ?)
        ''', ('John', 'Manager', '123-456-7890'))

except sqlite3.Error as e:
    print(f"Error: {e}")


In [37]:
with sqlite3.connect('database.db') as conn:
    cur = conn.cursor()
    # get the user whose name is 'John'
    cur.execute('SELECT * FROM users WHERE name = ?', ('John',))
    rows = cur.fetchall()
    for row in rows:
        print(row)
    # cur.execute('SELECT MAX(id) FROM users')
    # user_id = cur.fetchone()[0]
    # print(user_id)
    # cur.execute('SELECT  FROM users')
    # rows = cur.fetchall()
    # for row in rows:
    #     print(row)

(1, 'John', 'Manager', '123-456-7890')


In [25]:
# function to write to the database
def write_to_db(id, name, position, phone_number):
    try:
        with sqlite3.connect('database.db') as conn:
            cur = conn.cursor()
            cur.execute(
                '''
                CREATE TABLE IF NOT EXISTS users (
                    id TEXT PRIMARY KEY,
                    name TEXT,
                    position TEXT,
                    phone_number TEXT
                )
                '''
            )
            cur.execute('INSERT INTO users (id, name, position, phone_number) VALUES (?, ?, ?, ?)', (id, name, position, phone_number))
            conn.commit()
    except sqlite3.Error as e:
        print(f"Unable to write to database: {e}")

In [32]:
# function to get user info from the database using id
def get_user_info(id):
    try:
        with sqlite3.connect('database.db') as conn:
            cur = conn.cursor()
            cur.execute('SELECT * FROM users WHERE id = ?', (id,))
            row = cur.fetchone()
            return row
    except sqlite3.Error as e:
        print(f"Error: {e}")
        return None

In [13]:
user_inf = get_user_info(face_result)

In [31]:
id = 'mme173419'
name = 'Aanu'
position = 'Staff'
phone_number = '123-325-7890'
write_to_db(id, name, position, phone_number)

In [30]:
# drop the table
with sqlite3.connect('database.db') as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS users')

In [27]:
# cur.execute('DROP TABLE IF EXISTS users')

In [33]:
a = get_user_info('mme173419')

In [34]:
a

('mme173419', 'Aanu', 'Staff', '123-325-7890')